In [348]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# load secrets
load_dotenv()
import math
import json
from IPython.display import JSON


In [349]:
# iex cloud - api data request
iex_key = os.getenv("IEX_API_KEY")
url = 'https://cloud.iexapis.com/stable/ref-data/symbols?token=' + iex_key
response = requests.get(url)
data = response.json()
symbols_df = pd.DataFrame(data)
# print 
symbols_df.head()

,symbol,exchange,exchangeSuffix,exchangeName,exchangeSegment,exchangeSegmentName,name,date,type,iexId,region,currency,isEnabled,figi,cik,lei
0,A,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Agilent Technologies Inc.,2024-02-20,cs,IEX_46574843354B2D52,US,USD,True,BBG000C2V3D6,0001090872,QUIX8Y7A2WP0XRMW7G29
1,AA,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Alcoa Corp,2024-02-20,cs,IEX_4238333734532D52,US,USD,True,BBG00B3T3HD3,0001675149,549300T12EZ1F6PWWU29
2,AAA,ARCX,,Nyse Arca,ARCX,Nyse Arca,Investment Managers Series Trust II - AXS Firs...,2024-02-20,et,IEX_5030314338392D52,US,USD,True,BBG01B0JRCS6,0001587982,549300SU7ER9OFETRU41
3,AAAU,BATS,,Cboe Bzx U S Equities Exchange,BATS,Cboe Bzx U S Equities Exchange,Goldman Sachs Physical Gold ETF Trust - Goldma...,2024-02-20,et,IEX_474B433136332D52,US,USD,True,BBG00LPXX872,0001708646,None
4,AACG,XNAS,,Nasdaq All Markets,XNMS,Nasdaq Nms Global Market,ATA Creativity Global - ADR,2024-02-20,ad,IEX_44595A4C53392D52,US,USD,True,BBG000V2S3P6,0001420529,None


In [350]:
# NYSE and NASDAQ symbols only (filter)
filtered_df = symbols_df[symbols_df['exchange'].isin(['XNYS', 'XNAS'])]

# select cols
filtered_df = filtered_df[['symbol', 'exchange', 'exchangeName']]

# filter df
print(filtered_df.head())
data_size = len(filtered_df.index)
print()
print(f"Data size: {data_size}") # get length of dataframe

  symbol exchange                 exchangeName
0      A     XNYS  New York Stock Exchange Inc
1     AA     XNYS  New York Stock Exchange Inc
4   AACG     XNAS           Nasdaq All Markets
5   AACI     XNAS           Nasdaq All Markets
6  AACIU     XNAS           Nasdaq All Markets

Data size: 7791


In [351]:
batch_size = 100
# Assuming filtered_df is a DataFrame that contains a 'symbol' column.
# Calculate total batches
total_batches = math.ceil(len(filtered_df) / batch_size)
print(f"Total batches: {total_batches}")

combined_data = []

# function -> fetch market cap data in batches
def fetch_market_cap(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# Initialize a counter for completed batches
completed_batches = 0

# Define the length of the progress bar
progress_bar_length = 50

# batch processing - IMPORTANT (API efficiency)
for i in range(0, len(filtered_df), batch_size):
    batch_symbols = filtered_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_market_cap(batch_symbols, iex_key)  # Make sure iex_key is defined and valid
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        market_cap = batch_data.get(symbol, {}).get('quote', {}).get('marketCap', None)
        combined_data.append({'symbol': symbol, 'marketcap': market_cap})
    
    # Increment the completed batches counter
    completed_batches += 1

    # Calculate the progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # Create the progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # Print the progress bar with the percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Ensure the next print happens on the next line
print()

# convert combined data into a DataFrame
screener_df = pd.DataFrame(combined_data)

# join with the filtered_df - this adds exchange and exchangeName data
screener_df = screener_df.merge(filtered_df[['symbol', 'exchange', 'exchangeName']], on='symbol', how='left')

Total batches: 78
Progress: |██████████████████████████████████████████████████| 100.00% Complete


In [352]:
print(screener_df.head())
print()
print(len(screener_df.index))

  symbol     marketcap exchange                 exchangeName
0      A  3.930835e+10     XNYS  New York Stock Exchange Inc
1     AA  4.763415e+09     XNYS  New York Stock Exchange Inc
2   AACG  4.584605e+07     XNAS           Nasdaq All Markets
3   AACI  8.848655e+07     XNAS           Nasdaq All Markets
4  AACIU  8.848655e+07     XNAS           Nasdaq All Markets

7791


In [353]:
# drop NAs
screener_df.dropna(inplace=True)

# determins the count of droped NAs
print(f'New length: {len(screener_df)}')
print()
print(screener_df.head())

New length: 7683

  symbol     marketcap exchange                 exchangeName
0      A  3.930835e+10     XNYS  New York Stock Exchange Inc
1     AA  4.763415e+09     XNYS  New York Stock Exchange Inc
2   AACG  4.584605e+07     XNAS           Nasdaq All Markets
3   AACI  8.848655e+07     XNAS           Nasdaq All Markets
4  AACIU  8.848655e+07     XNAS           Nasdaq All Markets


In [354]:
screener_df.head(5)

,symbol,marketcap,exchange,exchangeName
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc
2,AACG,4.584605e+07,XNAS,Nasdaq All Markets
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets


In [355]:
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

quote_data_list = []

def fetch_quote_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# initialize a counter for completed batches
completed_batches = 0

# define the length of the progress bar
progress_bar_length = 50

"""scalable endpoint variable data request"""
for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_quote_data(batch_symbols, iex_key)
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        quote_data = batch_data.get(symbol, {}).get('quote', {})
        quote_data_list.append({
            'symbol': symbol, # KEY
            'latestPrice': quote_data.get('latestPrice', None),
            'close': quote_data.get('close', None), # 'close': 'latestPrice' or 'previousClose
            'previousClose': quote_data.get('previousClose', None),
            'extendedPrice': quote_data.get('extendedPrice', None), 
            'extendedChange': quote_data.get('extendedChange', None), 
            'extendedChangePercent': quote_data.get('extendedChangePercent', None), 
            'latestVolume': quote_data.get('latestVolume', None), # after hours volume
            'volume': quote_data.get('volume', None), # market hours volume
            'previousVolume': quote_data.get('previousVolume', None), # previous day volume
            'iexVolume': quote_data.get('iexVolume', None), # IEX volume
            'primaryExchange': quote_data.get('primaryExchange', None),
            'avgTotalVolume': quote_data.get('avgTotalVolume', None),
            'calculationPrice': quote_data.get('calculationPrice', None),
            'change': quote_data.get('change', None),
            'changePercent': quote_data.get('changePercent', None),
            'companyName': quote_data.get('companyName', None),})
    
    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
quote_df = pd.DataFrame(quote_data_list)

# MERGE DATA
screener_df = screener_df.merge(quote_df, on='symbol', how='left')

screener_df.head(5)

Total batches: 77


,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,latestVolume,volume,previousVolume,iexVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,134.14,134.14,134.84,134.14,0.0,0.0,1563909,1563909,1066838.0,43483.0,NEW YORK STOCK EXCHANGE INC.,1311345,close,-0.70,-0.00519,Agilent Technologies Inc.
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc,26.69,26.69,27.40,26.69,0.0,0.0,4711836,4711836,4686221.0,153167.0,NEW YORK STOCK EXCHANGE INC.,5761875,close,-0.71,-0.02591,Alcoa Corp
2,AACG,4.584605e+07,XNAS,Nasdaq All Markets,1.46,NaN,1.46,NaN,NaN,NaN,9242,9242,39354.0,0.0,NASDAQ,20623,previousclose,0.00,0.00000,ATA Creativity Global - ADR
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets,10.98,NaN,10.98,NaN,NaN,NaN,33,33,509.0,0.0,NASDAQ,14409,previousclose,0.00,0.00000,Armada Acquisition Corp I
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.90,NaN,10.90,NaN,NaN,NaN,1655,0,1655.0,0.0,NASDAQ,6983,previousclose,0.00,0.00000,Armada Acquisition Corp I - Units (1 Ord & 1/2...


In [356]:
def fetch_shares_outstanding_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=stats&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

# Assuming screener_df is a DataFrame containing the symbols
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

shares_outstanding_data_list = []

# initialize a counter for completed batches
completed_batches = 0
# define the length of the progress bar
progress_bar_length = 50

# Replace 'screener_df' with the actual DataFrame containing the symbols
for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_shares_outstanding_data(batch_symbols, iex_key)
    
    # Process and append data for each symbol in the batch
    for symbol in batch_symbols:
        stats_data = batch_data.get(symbol, {}).get('stats', {})
        shares_outstanding_data_list.append({
            'symbol': symbol,
            'sharesOutstanding': stats_data.get('sharesOutstanding', None),
            'avg10Volume': stats_data.get('avg10Volume', None),
            'avg30Volume': stats_data.get('avg30Volume', None),
        })

    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
shares_outstanding_df = pd.DataFrame(shares_outstanding_data_list)

# MERGE DATA
screener_df = screener_df.merge(shares_outstanding_df, on='symbol', how='left')

screener_df.head(5)

Total batches: 77


,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,...,iexVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName,sharesOutstanding,avg10Volume,avg30Volume
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,134.14,134.14,134.84,134.14,0.0,0.0,...,43483.0,NEW YORK STOCK EXCHANGE INC.,1311345,close,-0.70,-0.00519,Agilent Technologies Inc.,293039707,1249900,1311345
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc,26.69,26.69,27.40,26.69,0.0,0.0,...,153167.0,NEW YORK STOCK EXCHANGE INC.,5761875,close,-0.71,-0.02591,Alcoa Corp,178471908,5162225,5761875
2,AACG,4.584605e+07,XNAS,Nasdaq All Markets,1.46,NaN,1.46,NaN,NaN,NaN,...,0.0,NASDAQ,20623,previousclose,0.00,0.00000,ATA Creativity Global - ADR,31401405,42101,20623
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets,10.98,NaN,10.98,NaN,NaN,NaN,...,0.0,NASDAQ,14409,previousclose,0.00,0.00000,Armada Acquisition Corp I,8058884,461,14409
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.90,NaN,10.90,NaN,NaN,NaN,...,0.0,NASDAQ,6983,previousclose,0.00,0.00000,Armada Acquisition Corp I - Units (1 Ord & 1/2...,15000000,0,6983


In [ ]:
"""ADD other data here before starting the market cap calculation"""

In [357]:
# categorize market cap
def categorize_market_cap(x):
    if x < 300e6:  #  < 300 million
        return 'Micro-Cap'
    elif 300e6 <= x < 2e9:  # 300 million to 2 billion
        return 'Small-Cap'
    elif 2e9 <= x < 10e9:  # 2 billion to 10 billion
        return 'Mid-Cap'
    elif 10e9 <= x < 200e9:  # 10 billion to 200 billion
        return 'Large-Cap'
    elif x >= 200e9:  # >= 200 billion
        return 'Mega-Cap'
    else:
        return 'Unknown'  # handles negative or NaN values

# apply function to 'marketcap' column AND create new 'marketcapType' column
screener_df['marketcapType'] = screener_df['marketcap'].apply(categorize_market_cap)

screener_df.head(10)

,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,...,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName,sharesOutstanding,avg10Volume,avg30Volume,marketcapType
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,134.140,134.14,134.840,134.140,0.0,0.0,...,NEW YORK STOCK EXCHANGE INC.,1311345,close,-0.700,-0.00519,Agilent Technologies Inc.,293039707,1249900,1311345,Large-Cap
1,AA,4.763415e+09,XNYS,New York Stock Exchange Inc,26.690,26.69,27.400,26.690,0.0,0.0,...,NEW YORK STOCK EXCHANGE INC.,5761875,close,-0.710,-0.02591,Alcoa Corp,178471908,5162225,5761875,Mid-Cap
2,AACG,4.584605e+07,XNAS,Nasdaq All Markets,1.460,NaN,1.460,NaN,NaN,NaN,...,NASDAQ,20623,previousclose,0.000,0.00000,ATA Creativity Global - ADR,31401405,42101,20623,Micro-Cap
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets,10.980,NaN,10.980,NaN,NaN,NaN,...,NASDAQ,14409,previousclose,0.000,0.00000,Armada Acquisition Corp I,8058884,461,14409,Micro-Cap
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.900,NaN,10.900,NaN,NaN,NaN,...,NASDAQ,6983,previousclose,0.000,0.00000,Armada Acquisition Corp I - Units (1 Ord & 1/2...,15000000,0,6983,Micro-Cap
5,AACT,5.270000e+08,XNYS,New York Stock Exchange Inc,10.540,10.51,10.520,10.540,0.0,0.0,...,NEW YORK STOCK EXCHANGE INC.,214072,sip,0.020,0.00190,Ares Acquisition Corporation II - Class A,50000000,140291,214072,Small-Cap
6,AACT=,5.260000e+08,XNYS,New York Stock Exchange Inc,10.629,NaN,10.629,10.629,0.0,0.0,...,NEW YORK STOCK EXCHANGE INC.,4678,previousclose,0.000,0.00000,Ares Acquisition Corporation II - Units (1 Ord...,50000000,600,4678,Small-Cap
7,AADI,4.708965e+07,XNAS,Nasdaq All Markets,1.920,NaN,1.900,NaN,NaN,NaN,...,NASDAQ,227696,iexlasttrade,0.020,0.01053,Aadi Bioscience Inc,24525860,241834,227696,Micro-Cap
8,AADR,2.573550e+07,XNAS,Nasdaq All Markets,57.190,NaN,57.190,NaN,NaN,NaN,...,NASDAQ,730,previousclose,0.000,0.00000,Advisorshares Trust - AdvisorShares Dorsey Wri...,450000,1002,730,Micro-Cap
9,AAGR,4.490466e+07,XNAS,Nasdaq All Markets,0.776,NaN,0.829,NaN,NaN,NaN,...,NASDAQ,107901,iexlasttrade,-0.053,-0.06393,African Agriculture Holdings Inc,57866830,87929,107901,Micro-Cap


In [345]:
#print columns for screener_df
print(screener_df.columns)

Index(['symbol', 'marketcap', 'exchange', 'exchangeName', 'latestPrice',
       'close', 'previousClose', 'extendedPrice', 'extendedChange',
       'extendedChangePercent', 'latestVolume', 'volume', 'previousVolume',
       'iexVolume', 'primaryExchange', 'avgTotalVolume', 'calculationPrice',
       'change', 'changePercent', 'companyName', 'sharesOutstanding',
       'avg10Volume', 'avg30Volume', 'marketcapType'],
      dtype='object')


In [347]:
# create a columns filter
screener_cols = ['symbol', #mandatory
                 'marketcap',
                 'marketcapType', #mandatory
                 'close', #mandatory
                 'calculationPrice',
                 'change', #mandatory
                 'changePercent', #mandatory
                 'latestPrice', #mandatory
                 'extendedPrice',
                 'sharesOutstanding', #mandatory (substitute approximation for Float)
                 'latestVolume', #mandatory
                 'avg10Volume',
                 'avg30Volume']

screener_df_x = screener_df[screener_cols]

screener_df_x.head(2)

KeyError: "['extendedPrice'] not in index"

In [342]:
# # DEV - marketcap groupby count
# marketcap_counts = screener_df['marketcapType'].value_counts()

# # convert SERIES to DF
# marketcap_counts_df = marketcap_counts.reset_index()
# marketcap_counts_df.columns = ['marketcapType', 'count']

# # add percentage column
# total_count = marketcap_counts.sum()
# marketcap_counts_df['percentage'] = (marketcap_counts_df['count'] / total_count) * 100

# # print
# marketcap_counts_df

,marketcapType,count,percentage
0,Micro-Cap,3496,45.508982
1,Small-Cap,1932,25.149701
2,Mid-Cap,1248,16.245769
3,Large-Cap,936,12.184327
4,Mega-Cap,70,0.911221


In [343]:
# # DEV - checker
# total_count_CHECK = marketcap_counts_df['count'].sum()

# # if statement comparing the two values, return 'Data is correct' if they match, otherwise return 'Data is incorrect'
# if total_count_CHECK == len(screener_df):
#     print('Data lengths match! SUCCESS')
# else:    
#     print('Data mismatch! ERROR')
#     print()
#     print(f'Code checker: {total_count_CHECK}')
#     print(f'Data length: {len(screener_df)}')

Data lengths match! SUCCESS


END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END


In [287]:
# def print_available_stats_parameters(symbol, iex_key):
#     url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={iex_key}'
#     response = requests.get(url)
#     if response.status_code == 200:
#         data = response.json()
#         print(json.dumps(data, indent=4, sort_keys=True))
#     else:
#         print(f"Failed to fetch stats for {symbol}. Status code: {response.status_code}")

# pltr = 'PLTR'
# print_available_stats_parameters(pltr, iex_key)

In [288]:
def fetch_shares_outstanding_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=stats&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

# Assuming screener_df is a DataFrame containing the symbols
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

shares_outstanding_data_list = []

# initialize a counter for completed batches
completed_batches = 0
# define the length of the progress bar
progress_bar_length = 50

# Replace 'screener_df' with the actual DataFrame containing the symbols
for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_shares_outstanding_data(batch_symbols, iex_key)
    
    # Process and append data for each symbol in the batch
    for symbol in batch_symbols:
        stats_data = batch_data.get(symbol, {}).get('stats', {})
        shares_outstanding_data_list.append({
            'symbol': symbol,
            'sharesOutstanding': stats_data.get('sharesOutstanding', None),
            'avg10Volume': stats_data.get('avg10Volume', None),
            'avg30Volume': stats_data.get('avg30Volume', None),
        })

    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
shares_outstanding_df = pd.DataFrame(shares_outstanding_data_list)

Total batches: 77


In [289]:
# shares outstanding dataframe - 1 
shares_outstanding_df.head(10)

,symbol,sharesOutstanding,avg10Volume,avg30Volume
0,A,293039707,1249900,1311345
1,AA,178471908,5162225,5761875
2,AACG,31401405,42101,20623
3,AACI,8058884,461,14409
4,AACIU,15000000,0,6983
5,AACT,50000000,140291,214072
6,AACT=,50000000,600,4678
7,AADI,24525860,241834,227696
8,AADR,450000,1002,730
9,AAGR,57866830,87929,107901


In [290]:
# merge shares_outstanding_df with screener_df
screener_df = screener_df.merge(shares_outstanding_df[['symbol', 'sharesOutstanding', 'avg10Volume', 'avg30Volume']], on='symbol', how='left')

In [291]:
def fetch_deep_trades_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/deep/trades?symbols={symbols_str}&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

# Assuming screener_df is a DataFrame containing the symbols
batch_size = 100
# Example for illustrative purposes
# screener_df = pd.DataFrame({'symbol': ['AAPL', 'MSFT', 'GOOGL']})
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

deep_data_list = []

for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_deep_trades_data(batch_symbols, iex_key)
    
    # Process and infer volume data for each symbol in the batch
    for symbol in batch_symbols:
        if symbol in batch_data and batch_data[symbol]:
            volume = sum(trade['size'] for trade in batch_data[symbol])
            deep_data_list.append({
                'symbol': symbol,
                'volume': volume,
            })

# Convert the combined data into a DataFrame
iex_deep_df = pd.DataFrame(deep_data_list)

Total batches: 77


In [292]:
print(iex_deep_df.head(20)) # data check
print(len(iex_deep_df.index)) # data check

   symbol  volume
0       A   34993
1      AA  144756
2    AADI    2735
3    AAGR    6681
4     ACT    7302
5    ACTG    1169
6     ACV     313
7    ACVA   38457
8    ACWI   80905
9    ACWX    7910
10   ACXP     480
11   ADAG     203
12   ADAP   24986
13   ADBE     926
14  AGM-F     100
15   AGMH     271
16   AGNC  155760
17  AGNCL     241
18  AGNCM      44
19   ALCE    3153
469


In [293]:
print(f"screener_df column count: {len(screener_df.columns)}") # col count
screener_df = screener_df.merge(shares_outstanding_df, on='symbol', how='left')

screener_df column count: 15


In [294]:
# merge shares_outstanding_df AND iex_deep_df with screener_df
screener_df = screener_df.merge(iex_deep_df, on='symbol', how='left')
print(f"screener_df column count: {len(screener_df.columns)}") # col count

screener_df.head(24) # data check

screener_df column count: 19


,symbol,marketcap,exchange,exchangeName,marketcapType,latestPrice,previousClose,extendedPrice,extendedChangePercent,extendedPriceTime,latestVolume,companyName,sharesOutstanding_x,avg10Volume_x,avg30Volume_x,sharesOutstanding_y,avg10Volume_y,avg30Volume_y,volume
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,Large-Cap,134.140,134.840,134.140,0.00000,1.708463e+12,1563902,Agilent Technologies Inc.,293039707,1249900,1311345,293039707,1249900,1311345,34993.0
1,AA,4.765200e+09,XNYS,New York Stock Exchange Inc,Mid-Cap,26.700,27.400,26.700,0.00000,1.708463e+12,4687957,Alcoa Corp,178471908,5162225,5761875,178471908,5162225,5761875,144756.0
2,AACG,4.584605e+07,XNAS,Nasdaq All Markets,Micro-Cap,1.460,1.460,NaN,NaN,NaN,9242,ATA Creativity Global - ADR,31401405,42101,20623,31401405,42101,20623,NaN
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets,Micro-Cap,10.980,10.980,NaN,NaN,NaN,33,Armada Acquisition Corp I,8058884,461,14409,8058884,461,14409,NaN
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,Micro-Cap,10.900,10.900,NaN,NaN,NaN,1655,Armada Acquisition Corp I - Units (1 Ord & 1/2...,15000000,0,6983,15000000,0,6983,NaN
5,AACT,5.265000e+08,XNYS,New York Stock Exchange Inc,Small-Cap,10.530,10.520,10.520,-0.00095,1.708128e+12,307792,Ares Acquisition Corporation II - Class A,50000000,140291,214072,50000000,140291,214072,NaN
6,AACT=,5.260000e+08,XNYS,New York Stock Exchange Inc,Small-Cap,10.629,10.629,10.629,0.00000,1.708042e+12,600,Ares Acquisition Corporation II - Units (1 Ord...,50000000,600,4678,50000000,600,4678,NaN
7,AADI,4.708965e+07,XNAS,Nasdaq All Markets,Micro-Cap,1.920,1.900,NaN,NaN,NaN,169466,Aadi Bioscience Inc,24525860,241834,227696,24525860,241834,227696,2735.0
8,AADR,2.573550e+07,XNAS,Nasdaq All Markets,Micro-Cap,57.190,57.190,NaN,NaN,NaN,6977,Advisorshares Trust - AdvisorShares Dorsey Wri...,450000,1002,730,450000,1002,730,NaN
9,AAGR,4.490466e+07,XNAS,Nasdaq All Markets,Micro-Cap,0.776,0.829,NaN,NaN,NaN,134598,African Agriculture Holdings Inc,57866830,87929,107901,57866830,87929,107901,6681.0


In [295]:
# # My Watchlist
# watchlist = ['PLTR', 'TSLA', 'NOW', 'SNOW','FB', 'NVDA', 'PYPL', 'ADBE', 'NFLX']
# watchlist_df = screener_df[screener_df['symbol'].isin(watchlist)]

# watchlist_df

In [296]:
import requests
import pandas as pd
import math

def fetch_market_volume(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    # Corrected URL with 'quote' as the type to include volume information
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

# Assuming screener_df is a DataFrame containing the symbols and iex_key is defined
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

market_volume_data_list = []

completed_batches = 0
progress_bar_length = 50

for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_market_volume(batch_symbols, iex_key)
    
    for symbol in batch_symbols:
        quote_data = batch_data.get(symbol, {}).get('quote', {})
        market_volume_data_list.append({
            'symbol': symbol,
            'latestVolume': quote_data.get('latestVolume', None),
            # You can add more fields here as needed
        })

    completed_batches += 1
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Ensure the next print happens on the next line
print()

# Convert the combined data into a DataFrame
market_volume_df = pd.DataFrame(market_volume_data_list)

Total batches: 77
Progress: |██████████████████████████████████████████████████| 100.00% Complete


In [297]:
# print
market_volume_df.head(10)

,symbol,latestVolume
0,A,1563902
1,AA,4688057
2,AACG,9242
3,AACI,33
4,AACIU,1655
5,AACT,307792
6,AACT=,600
7,AADI,169466
8,AADR,6977
9,AAGR,134598


In [298]:
# merger market_volume_df with screener_df
screener_df = screener_df.merge(market_volume_df, on='symbol', how='left')

In [299]:
# get lastUpdated (beta)
# get other data if needed from this endpoint
# verify accuracy
# learn more about volume data aquisition and theory

In [300]:
screener_df.head(10)

,symbol,marketcap,exchange,exchangeName,marketcapType,latestPrice,previousClose,extendedPrice,extendedChangePercent,extendedPriceTime,latestVolume_x,companyName,sharesOutstanding_x,avg10Volume_x,avg30Volume_x,sharesOutstanding_y,avg10Volume_y,avg30Volume_y,volume,latestVolume_y
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,Large-Cap,134.140,134.840,134.140,0.00000,1.708463e+12,1563902,Agilent Technologies Inc.,293039707,1249900,1311345,293039707,1249900,1311345,34993.0,1563902
1,AA,4.765200e+09,XNYS,New York Stock Exchange Inc,Mid-Cap,26.700,27.400,26.700,0.00000,1.708463e+12,4687957,Alcoa Corp,178471908,5162225,5761875,178471908,5162225,5761875,144756.0,4688057
2,AACG,4.584605e+07,XNAS,Nasdaq All Markets,Micro-Cap,1.460,1.460,NaN,NaN,NaN,9242,ATA Creativity Global - ADR,31401405,42101,20623,31401405,42101,20623,NaN,9242
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets,Micro-Cap,10.980,10.980,NaN,NaN,NaN,33,Armada Acquisition Corp I,8058884,461,14409,8058884,461,14409,NaN,33
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,Micro-Cap,10.900,10.900,NaN,NaN,NaN,1655,Armada Acquisition Corp I - Units (1 Ord & 1/2...,15000000,0,6983,15000000,0,6983,NaN,1655
5,AACT,5.265000e+08,XNYS,New York Stock Exchange Inc,Small-Cap,10.530,10.520,10.520,-0.00095,1.708128e+12,307792,Ares Acquisition Corporation II - Class A,50000000,140291,214072,50000000,140291,214072,NaN,307792
6,AACT=,5.260000e+08,XNYS,New York Stock Exchange Inc,Small-Cap,10.629,10.629,10.629,0.00000,1.708042e+12,600,Ares Acquisition Corporation II - Units (1 Ord...,50000000,600,4678,50000000,600,4678,NaN,600
7,AADI,4.708965e+07,XNAS,Nasdaq All Markets,Micro-Cap,1.920,1.900,NaN,NaN,NaN,169466,Aadi Bioscience Inc,24525860,241834,227696,24525860,241834,227696,2735.0,169466
8,AADR,2.573550e+07,XNAS,Nasdaq All Markets,Micro-Cap,57.190,57.190,NaN,NaN,NaN,6977,Advisorshares Trust - AdvisorShares Dorsey Wri...,450000,1002,730,450000,1002,730,NaN,6977
9,AAGR,4.490466e+07,XNAS,Nasdaq All Markets,Micro-Cap,0.776,0.829,NaN,NaN,NaN,134598,African Agriculture Holdings Inc,57866830,87929,107901,57866830,87929,107901,6681.0,134598


In [309]:
# save to csv so you don't have to load everytime to reset the data
screener_df.to_csv('screener_data.csv', index=False)

RESET

In [ ]:
# # drop all but the symbol and marketcap columns
# screener_df = screener_df[['symbol', 'marketcap']]

# screener_df.head()

POST-RESET LOAD

In [314]:
# load csv data state
screener_df = pd.read_csv('screener_data.csv')

screener_df.head()

,symbol,marketcap,exchange,exchangeName,marketcapType,latestPrice,previousClose,extendedPrice,extendedChangePercent,extendedPriceTime,latestVolume_x,companyName,sharesOutstanding_x,avg10Volume_x,avg30Volume_x,sharesOutstanding_y,avg10Volume_y,avg30Volume_y,volume,latestVolume_y
0,A,3.930835e+10,XNYS,New York Stock Exchange Inc,Large-Cap,134.14,134.84,134.14,0.0,1.708463e+12,1563902,Agilent Technologies Inc.,293039707,1249900,1311345,293039707,1249900,1311345,34993.0,1563902
1,AA,4.765200e+09,XNYS,New York Stock Exchange Inc,Mid-Cap,26.70,27.40,26.70,0.0,1.708463e+12,4687957,Alcoa Corp,178471908,5162225,5761875,178471908,5162225,5761875,144756.0,4688057
2,AACG,4.584605e+07,XNAS,Nasdaq All Markets,Micro-Cap,1.46,1.46,NaN,NaN,NaN,9242,ATA Creativity Global - ADR,31401405,42101,20623,31401405,42101,20623,NaN,9242
3,AACI,8.848655e+07,XNAS,Nasdaq All Markets,Micro-Cap,10.98,10.98,NaN,NaN,NaN,33,Armada Acquisition Corp I,8058884,461,14409,8058884,461,14409,NaN,33
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,Micro-Cap,10.90,10.90,NaN,NaN,NaN,1655,Armada Acquisition Corp I - Units (1 Ord & 1/2...,15000000,0,6983,15000000,0,6983,NaN,1655


DATA FINALIZATION

In [301]:
# group_df_cols = ['symbol',
#                  'marketcapType',
#                  'marketcap',
#                  'previousClose',
#                  'extededPrice',
#                  'extendedChangePercent',
#                  'sharesOutstanding',
#                  'latestVolume',
#                  'avg10Volume',
#                  'avg30Volume']

In [302]:
# screener_df.drop(columns=['avg10Volume'], inplace=True)

In [303]:
# screener_df.head(1)

In [304]:
# # filtered dataframe for selected stocks (TSLA, PLTR, SNOW)
# selected_stocks = screener_df[(screener_df['symbol'] == 'TSLA') | (screener_df['symbol'] == 'PLTR') | (screener_df['symbol'] == 'SNOW')]
# # fix index
# selected_stocks.reset_index(drop=True, inplace=True)

In [305]:
# # look at filterd dataframe
# selected_stocks

In [306]:
# # drop columns other than 'sybol', 'marketcap', 'exchange', 'exchangeName', 'marketcapType'
# screener_df = screener_df[['symbol', 'marketcap', 'exchange', 'exchangeName', 'marketcapType',]]

# screener_df.head(10)

In [307]:
# # DEV - API endpoing parameter availability. 
# schema_base_test_url = 'https://cloud.iexapis.com/stable'
# test_symbol = 'PLTR' # test ticker symbol
# schema_test_url = f"{schema_base_test_url}/stock/{test_symbol}/quote?schema=true&token={iex_key}"
# # GET request
# schema_test_response = requests.get(schema_test_url)

# print(json.dumps(schema_test_response.json(), indent=4))


In [308]:
# # shares outstanding
# symbol = 'PLTR'
# so_test_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats/sharesOutstanding?token={iex_key}'

# response = requests.get(so_test_url)
# if response.status_code == 200:
#     shares_outstanding = response.json()
#     print(f'Average Shares Outstanding for {symbol}: {shares_outstanding}')
# else:
#     print(f'Failed to fetch data for {symbol}. Status code: {response.status_code}')


IDEAS
1. Robust value score (RBV)
2. create data frames for each market cap
3. create a function that does math and querying based on rules / conditions
4. apply function to the market cap data sets
5. Look at other STATS now that the API is connected for the sharesOutstanding

NOTES
1. Float
1. Free Flt Mkt Cap
1. Free Flt
1. Shares outstanding

MANDATORY FIELDS
1. 